# Влияние разных факторов на количество лайков на странице ВКонтакте.

Цель данного проекта заключается в том, чтобы выяснить, какой из трех фактором является важнейшним при публикации записи в социальной сети ВКонтакте для получения наибольшего количества лайков. Наши рассматриваемые факторы: время суток публикации, день недели или промежуток между постами. Для ответа на данный вопрос я спарсил информацию о постах (*файл posts_parser.py*) с [личной страницы Сергея Семёновича Собянина во ВКонтакте](https://vk.com/mossobyanin). На странице Сергея Семёновича было опубликовано около семи тысяч записей. Значит, результаты данного исследования могут считаться более или менее репрезентативными.

In [185]:
import sqlite3
import pandas as pd

Импорт необходимых библиотек - *sqlite3* для работы с SQL-запросами и *pandas* для дальнейшего анализа зависимости между факторами и количеством лайков.

In [186]:
connection = sqlite3.connect('Sobyanin_posts_data.db')

Создания подключения к базе данных с информацией о постах.

## Время суток и количество лайков.

In [187]:
query1 = '''
    SELECT time_of_day, SUM(likes_count) AS total_likes
    FROM posts
    GROUP BY time_of_day
    ORDER BY total_likes DESC
'''

result1 = connection.execute(query1).fetchall()
lresult1 = list(result1)

for row in result1:
    print(f"Время суток: {row[0]}, Количество лайков: {row[1]}")

Время суток: day, Количество лайков: 1822528
Время суток: evening, Количество лайков: 813593
Время суток: morning, Количество лайков: 524972
Время суток: night, Количество лайков: 16752


В данном запросе мы агрегируем данные по столбцу *time_of_day* и вычисляем суммарное количество лайков для каждого времени дня. 
Выводим результат по убыванию столбца *total_likes*. Вывод: больше всего лайков проставили на посты, которые были выложены днем.

## День недели и количество лайков.

In [188]:
query2 = '''
    SELECT day_of_week, SUM(likes_count) AS total_likes
    FROM posts
    GROUP BY day_of_week
    ORDER BY total_likes DESC
'''

result2 = connection.execute(query2).fetchall()
lresult2 = list(result2)

for row in result2:
    print(f"День недели: {row[0]}, Количество лайков: {row[1]}")

День недели: friday, Количество лайков: 501458
День недели: tuesday, Количество лайков: 496997
День недели: thursday, Количество лайков: 472433
День недели: wednesday, Количество лайков: 469814
День недели: saturday, Количество лайков: 421331
День недели: monday, Количество лайков: 415579
День недели: sunday, Количество лайков: 400233


Агрегируем данные, но уже по столбцу *day_of_week*. Можем заметить, что разница в количестве лайков не такая колоссальная, как в случае с временем суток публикации. Так или иначе, больше всего лайков было проставлено на посты, опубликованные в пятницу.

## Интервал (в днях) между постами и количество лайков.

In [189]:
query3 = """
WITH lagged_posts AS (
    SELECT date(full_date) AS full_date,
           likes_count,
           LAG(date(full_date)) OVER (ORDER BY date(full_date)) AS lag_full_date
    FROM posts
)
SELECT days_diff, SUM(likes_count) AS total_likes
FROM (
    SELECT IFNULL(julianday(full_date) - julianday(lag_full_date), -1) AS days_diff,
           likes_count
    FROM lagged_posts
)
WHERE days_diff != -1
GROUP BY days_diff
ORDER BY total_likes DESC;
"""

result3 = connection.execute(query3).fetchall()
lresult3 = list(result3)

for row in result3:
    print(f"Интервал (в днях): {row[0]}, Количество лайков: {row[1]}")

Интервал (в днях): 0.0, Количество лайков: 1908477
Интервал (в днях): 1.0, Количество лайков: 1124839
Интервал (в днях): 2.0, Количество лайков: 86596
Интервал (в днях): 3.0, Количество лайков: 34970
Интервал (в днях): 4.0, Количество лайков: 11594
Интервал (в днях): 5.0, Количество лайков: 1834
Интервал (в днях): 7.0, Количество лайков: 1008
Интервал (в днях): 48.0, Количество лайков: 267


Интервал (в днях) представляет собой разницу между датой текущего поста и предыдущего. Например, если для определенного интервала (например, 1.0) у нас есть суммарное количество лайков 1124839, это означает, что в среднем между постами, размещенными с интервалом в 1 день, получается примерно 1124839 лайков.

Агрегируем данные по интервалу (в днях) между постами. Видим, что чем чаще выходят посты (интервал между постами 0 дней, 1 день, 2 дня), тем больше лайков на публикациях. Делаем вывод: частые публикации позитивно влияют на активность аудитории.

In [190]:
connection.close()

Прерывание соединения с базой данных.

## Определение наиболее важного фактора.

Так как столбцы *time_of_day* и *day_of_week* хранят в себе категориальные значения, для определения корреляции между этими факторами и количеством лайков будет использоваться преобразование категориальных переменных в числовой формат, который можно использовать для расчета корреляции с непрерывными переменными. В процессе преобразования *time_of_day* и *day_of_week* каждое уникальное значение заменяется числовым кодом.

Для определения зависимости будет использоваться библиотека *pandas* и ее метод *[corr](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.corr.html)*.

In [191]:
df1 = pd.DataFrame(lresult1, columns=['time_of_day', 'likes_count'])
df1['time_of_day_code'] = df1['time_of_day'].astype('category').cat.codes


df2 = pd.DataFrame(lresult2, columns=['day_of_week', 'likes_count'])
df2['day_of_week_code'] = df2['day_of_week'].astype('category').cat.codes

df3 = pd.DataFrame(lresult3, columns=['days_interval', 'likes_count'])

In [192]:
df1

,time_of_day,likes_count,time_of_day_code
0,day,1822528,0
1,evening,813593,1
2,morning,524972,2
3,night,16752,3


In [193]:
df2

,day_of_week,likes_count,day_of_week_code
0,friday,501458,0
1,tuesday,496997,5
2,thursday,472433,4
3,wednesday,469814,6
4,saturday,421331,2
5,monday,415579,1
6,sunday,400233,3


In [194]:
correlation1 = df1['time_of_day_code'].corr(df1['likes_count'])
correlation2 = df2['day_of_week_code'].corr(df2['likes_count'])
correlation3 = df3['days_interval'].corr(df3['likes_count'])

print(f"Корреляция между временем суток и количеством лайков: {correlation1}")
print(f"Корреляция между днем недели и количеством лайков: {correlation2}")
print(f"Корреляция между интервалом между постами (в днях) и количеством лайков: {correlation3}")

Корреляция между временем суток и количеством лайков: -0.9687133076489198
Корреляция между днем недели и количеством лайков: 0.22350556848687878
Корреляция между интервалом между постами (в днях) и количеством лайков: -0.3238546775522475


## Выводы

* Корреляция между time_of_day и likes_count равна -0.9687. Значение близкое к -1 указывает на сильную обратную зависимость между временем дня и количеством лайков. Исходя из итоговой таблицы: больше лайков получают дневные посты, меньше - ночные.

* Корреляция между day_of_week и likes_count равна 0.2235. Значение близкое к 0 указывает на слабую положительную связь между днем недели и количеством лайков. Это может указывать на то, что количество лайков слабо зависит от дня недели.

* Корреляция между days_interval и likes_count равна -0.3239. Значение близкое к -1 указывает на слабую обратную зависимость между интервалом дней и количеством лайков. Так или иначе, можно заметить, что частые посты получают гораздо больше лайков (см. предыдущий пункт).

**Самым важным их всех трех факторов можно считать время суток, в которое публикуется пост. Для получения наибольшего количества лайков лучше всего опубликовывать запись днем.**